Data Preprocessing

In [1]:
#installing the natural language tool kit for normalization of the dataset
%pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [2]:
#imporiting the required dependencies
from collections import defaultdict
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/dipp3r/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dipp3r/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/dipp3r/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/dipp3r/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [27]:
data_df1 = pd.read_csv("../dataset/dataset1.csv",sep=",",header=0,quotechar='"')
data_df2 = pd.read_csv("../dataset/dataset2.csv",sep=",",header=0,quotechar='"')
data_df2 = data_df2[:4700]
data_df2.reset_index(drop=True)
data_df2.index += 1234
data_df = pd.concat([data_df1, data_df2], axis=0)
print(data_df)

                                            review_text  F  BR  AU  FI  IR  A  \
0     Make it like better with a giant pig bigger th...  0   0   0   0   0  0   
1     These screens are small enough without crowdin...  0   0   0   0   0  0   
2     Dear Rovio; If you absolutely must continue tr...  0   0   0   0   1  0   
3         App crashes when new power up notice pops up.  0   1   0   0   0  0   
4     It would be nice to have an update that didnt ...  0   1   0   0   0  0   
...                                                 ... ..  ..  ..  ..  .. ..   
5929  This  then stopped working completely on our t...  0   0   0   0   0  0   
5930              Add history and background of drivers  1   0   0   0   0  0   
5931  Wish Grab Car would be available again in prov...  0   0   0   0   0  1   
5932  Auto delete Recently, when I reinstall. Its ca...  0   0   0   0   0  0   
5933                              Can register my gmail  0   0   1   0   0  0   

      L  LF  MN  O  PE  SC 

In [28]:
# max and minimum length review
max_length_review = data_df["review_text"].iloc[data_df["review_text"].str.len().idxmax()]
print(len(max_length_review))

min_length_review = data_df["review_text"].iloc[data_df["review_text"].str.len().idxmin()]
print(len(min_length_review))

523
1


In [29]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# List of text to tokenize
texts = ["This is the first text.", "And here is the second text.", "Finally, the third text."]

# Tokenize the list of text with max_length=128
tokenized_inputs = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="tf")

# Print the tokenized inputs
print("Input IDs:\n", tokenized_inputs['input_ids'])
print("\nAttention Masks:\n", tokenized_inputs['attention_mask'])


Input IDs:
 tf.Tensor(
[[ 101 2023 2003 1996 2034 3793 1012  102    0]
 [ 101 1998 2182 2003 1996 2117 3793 1012  102]
 [ 101 2633 1010 1996 2353 3793 1012  102    0]], shape=(3, 9), dtype=int32)

Attention Masks:
 tf.Tensor(
[[1 1 1 1 1 1 1 1 0]
 [1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 0]], shape=(3, 9), dtype=int32)


In [30]:
list(data_df["review_text"]) 

['Make it like better with a giant pig bigger than king pig.',
 'These screens are small enough without crowding in more stuff.Tuck it in with the rest of the add ons I never use and please get it out of my face!',
 'Dear Rovio; If you absolutely must continue trying to squeeze dollars from your customers this way, at least have the decency to include an opt out for those like myself who are absolutely NOT going to spend an amount equal to what I paid for the app in the first place just to have some gimmicky and expendable in game item.',
 'App crashes when new power up notice pops up.',
 'It would be nice to have an update that didnt crash the app.',
 'App crashes with new release.',
 'I have had no problems with any of the versions so far.',
 'Please bring back the trajectory line or at least make it to where you can turn it on or off.',
 'This is not worth buying it will crash right from the beginning.',
 'I will say you should make shock so close to the other buttons because I like

In [31]:
import re

def custom_tokenizer(text):
    # Define regular expression pattern to identify contractions
    contraction_pattern = re.compile(r"\b(?:didn't|doesn't|don't|hadn't|hasn't|haven't|isn't|shouldn't|wasn't|weren't|won't|wouldn't)\b|\b(?:I|he|she|it|you|we|they)'(?:d|ll|m|re|s|ve)\b|\b\w+(?:n't)\b")
    words = text.split()
    # Process each token and separate punctuations
    final_tokens = []
    for token in words:
        # Remove leading and trailing spaces
        token = token.strip()
        if token:
            if contraction_pattern.match(token):
                final_tokens.append(token)
            # Tokenize punctuations
            else:
                punctuation_tokens = re.findall(r"\b\w+\b|[\.,!?;]", token)
                final_tokens.extend(punctuation_tokens)
    return final_tokens



In [32]:
from transformers import BertTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from collections import defaultdict
from nltk.corpus import wordnet as wn
import tensorflow as tf

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define NLTK resources
lemmatizer = WordNetLemmatizer()
english_stopwords = set(stopwords.words('english'))
NEGATION_SET = {"aren't", "won't", 'nor',  "doesn't", "weren't", "shan't", 'no', 'hadn', "couldn't", "don't", 'needn', "shouldn't",  'shouldn', 'mustn', "wasn't", 'isn', "needn't", 'ain', "hadn't", "haven't",  "didn't", 'didn', 'couldn', 'weren',  "mustn't", 'hasn', 'mightn', 'wouldn', "wouldn't",  "isn't", 'not', "hasn't",  'wasn',  'aren', 'haven', "mightn't",  'doesn',  'won', 'don'}
english_stopwords = english_stopwords - NEGATION_SET
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

def preprocess_comments_bert(reviews, max_length=64):

  # Convert to lowercase
  reviews = [review.lower() for review in reviews]

  # Tokenize using NLTK word tokenizer
  tokenized_reviews = [custom_tokenizer(review) for review in reviews]


  # Remove stopwords and perform lemmatization
  filtered_tokens = [[lemmatizer.lemmatize(word, tag_map[tag[0]]) for word, tag in pos_tag(review) 
                      if (word.isalpha() or word in NEGATION_SET) and word not in english_stopwords] for review in tokenized_reviews]
  
  print(filtered_tokens)
  
  tokenized_inputs = tokenizer(
      [' '.join(review) for review in filtered_tokens],
      add_special_tokens=True,
      padding=True, 
      truncation=True, 
      max_length=max_length, 
      return_tensors="tf"
    )
  
  return tokenized_inputs


In [33]:
review_inputs = preprocess_comments_bert(list(data_df["review_text"]))
review_inputs

[['make', 'like', 'well', 'giant', 'pig', 'big', 'king', 'pig'], ['screen', 'small', 'enough', 'without', 'crowd', 'stuff', 'tuck', 'rest', 'add', 'ons', 'never', 'use', 'please', 'get', 'face'], ['dear', 'rovio', 'absolutely', 'must', 'continue', 'try', 'squeeze', 'dollar', 'customer', 'way', 'least', 'decency', 'include', 'opt', 'like', 'absolutely', 'not', 'go', 'spend', 'amount', 'equal', 'pay', 'app', 'first', 'place', 'gimmicky', 'expendable', 'game', 'item'], ['app', 'crash', 'new', 'power', 'notice', 'pop'], ['would', 'nice', 'update', 'didnt', 'crash', 'app'], ['app', 'crash', 'new', 'release'], ['no', 'problem', 'version', 'far'], ['please', 'bring', 'back', 'trajectory', 'line', 'least', 'make', 'turn'], ['not', 'worth', 'buy', 'crash', 'right', 'beginning'], ['say', 'make', 'shock', 'close', 'button', 'like', 'use', 'bird', 'provide'], ['need', 'change'], ['similarly', 'every', 'single', 'app', 'get', 'update', 'beautiful', 'look', 'one', 'not', 'update'], ['really', 'fix']

{'input_ids': <tf.Tensor: shape=(5934, 64), dtype=int32, numpy=
array([[  101,  2191,  2066, ...,     0,     0,     0],
       [  101,  3898,  2235, ...,     0,     0,     0],
       [  101,  6203, 20996, ...,     0,     0,     0],
       ...,
       [  101,  4299,  6723, ...,     0,     0,     0],
       [  101,  8285,  3972, ...,     0,     0,     0],
       [  101,  4236, 20917, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(5934, 64), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(5934, 64), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32

In [34]:
#Saving the DataFrame as in serialized format to preserve the properties of the data structures(Tensor)

import pickle

#Define the file path where you want to save the serialized DataFrame
file_path = 'output/serialized_review_dataframe.pkl'

# Step 3: Save the DataFrame using pickle
with open(file_path, 'wb') as f:
    pickle.dump(review_inputs, f)

print("DataFrame saved as a serialized file:", file_path)

DataFrame saved as a serialized file: output/serialized_review_dataframe.pkl
